In [2]:
import requests
import pandas as pd
import json

API_KEY = 'yC64acO6xj53iW03VTdPX6QZxY5Y7najMElljT50'
HEADERS = {'x-api-key': API_KEY}


In [3]:
# Load character mapping
with open('character_mapping.json', 'r', encoding='utf-8') as f:
    character_map = json.load(f)

# Load weapon mapping
with open('weapon_mapping.json', 'r', encoding='utf-8') as f:
    weapon_map = json.load(f)

with open('item_mapping.json', 'r', encoding='utf-8') as f:
    item_map = json.load(f)
    
with open('augment_mapping.json', 'r', encoding='utf-8') as f:
    augment_map = json.load(f)


In [9]:
game_id = 46678263

# Function to get the game data for all players in a game
def get_game_data(game_id):
    url = f'https://open-api.bser.io/v1/games/{game_id}'
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to fetch game data: {response.status_code} - {response.text}")
        return None

    data = response.json()

    # Initialize list to store player data
    players_data = []

    # Iterate through players in the 'userGames' list
    for player in data.get('userGames', []):
        # Extract raw data values from the API response
        user_num = player.get('userNum', '')
        nickname = player.get('nickname', '')
        game_id = player.get('gameId', '')
        season_id = player.get('seasonId', '')
        game_Rank = player.get('gameRank', '')
        character_Num = player.get('characterNum', '')
        team_kill = player.get('teamKill','')
        kill = player.get('playerKill', '')
        death = player.get('playerDeaths', '')
        assist = player.get('playerAssistant','')
        dmg_player = player.get('damageToPlayer','')
        dmg_monster = player.get('damageToMonster','')

        # Get equipment data (raw values, no mapping)
        equipment = player.get('equipment', {})
        item_weapon = equipment.get('0', 'Unknown')
        item_chest = equipment.get('1', 'Unknown')
        item_head = equipment.get('2', 'Unknown')
        item_arm = equipment.get('3', 'Unknown')
        item_leg = equipment.get('4', 'Unknown')
        
        # Get trait data (raw values, no mapping)
        first_sub = player.get('traitFirstSub', [])
        second_sub = player.get('traitSecondSub', [])

        trait_main1 = first_sub[0] if len(first_sub) > 0 else ""
        trait_main2 = first_sub[1] if len(first_sub) > 1 else ""
        trait_sub1 = second_sub[0] if len(second_sub) > 0 else ""
        trait_sub2 = second_sub[1] if len(second_sub) > 1 else ""

        # Append player data to the list
        players_data.append({
            'userNum': user_num,
            'nickname': nickname,
            'gameId': game_id,
            'seasonId': season_id,
            'gameRank': game_Rank,
            'characterNum': character_Num,
            'teamKill': team_kill,
            'kill': kill,
            'death': death,
            'assist': assist,
            'dmgPlayer': dmg_player,
            'dmgMonster': dmg_monster,
            'itemWeapon': item_weapon,
            'itemChest': item_chest,
            'itemHead': item_head,
            'itemArm': item_arm,
            'itemLeg': item_leg,
            'Trait_Main1': trait_main1,
            'Trait_Main2': trait_main2,
            'Trait_Sub1': trait_sub1,
            'Trait_Sub2': trait_sub2
        })

    return players_data

# Call the function and get the game data
game_data = get_game_data(game_id)

# Print the raw data (including items and traits without mapping)
if game_data:
    for player in game_data:
        print(f"Player: {player['nickname']} (UserNum: {player['userNum']})")
        print(f"  Game ID: {player['gameId']} | Season ID: {player['seasonId']} | Rank: {player['gameRank']}")
        print(f"  Character Num: {player['characterNum']}")
        print(f"  Team Kill: {player['teamKill']} | Kills: {player['kill']} | Deaths: {player['death']} | Assists: {player['assist']}")
        print(f"  Damage to Players: {player['dmgPlayer']} | Damage to Monsters: {player['dmgMonster']}")
        print(f"  Equipment: Weapon: {player['itemWeapon']}, Chest: {player['itemChest']}, Head: {player['itemHead']}, Arm: {player['itemArm']}, Leg: {player['itemLeg']}")
        print(f"  Traits: Main1: {player['Trait_Main1']}, Main2: {player['Trait_Main2']}, Sub1: {player['Trait_Sub1']}, Sub2: {player['Trait_Sub2']}")
        print("-" * 40)
else:
    print("No data available.")


Player: 尋風追憶 (UserNum: 5359969)
  Game ID: 46678263 | Season ID: 31 | Rank: 7
  Character Num: 29
  Team Kill: 12 | Kills: 3 | Deaths: 2 | Assists: 7
  Damage to Players: 10909 | Damage to Monsters: 31758
  Equipment: Weapon: 111501, Chest: 202404, Head: 201507, Arm: 705608, Leg: 204419
  Traits: Main1: 7010701, Main2: 7310301, Sub1: 7010501, Sub2: 7010601
----------------------------------------
Player: 뎁미니스트 (UserNum: 5294646)
  Game ID: 46678263 | Season ID: 31 | Rank: 5
  Character Num: 65
  Team Kill: 15 | Kills: 2 | Deaths: 1 | Assists: 11
  Damage to Players: 13044 | Damage to Monsters: 49417
  Equipment: Weapon: 102412, Chest: 202506, Head: 201522, Arm: 205401, Leg: 204417
  Traits: Main1: 7110801, Main2: 7210801, Sub1: 7110101, Sub2: 7110601
----------------------------------------
Player: 범지 (UserNum: 4911433)
  Game ID: 46678263 | Season ID: 31 | Rank: 4
  Character Num: 47
  Team Kill: 15 | Kills: 5 | Deaths: 4 | Assists: 6
  Damage to Players: 16738 | Damage to Monsters: 8

In [7]:
game_id = 46678263

# Function to get the game data for all players in a game
def get_game_data(game_id):
    url = f'https://open-api.bser.io/v1/games/{game_id}'
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to fetch game data: {response.status_code} - {response.text}")
        return None

    data = response.json()

    # Initialize list to store player data
    players_data = []

    # Define valid weapon mastery IDs (as strings)
    weapon_mastery_ids = {
        "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11",
        "12", "13", "14", "15", "16", "17", "18", "19", "20",
        "21", "22", "23", "24", "25"
    }
        
    # Iterate through players in the 'userGames' list
    for player in data.get('userGames', []):
        user_num = player.get('userNum', '')
        nickname = player.get('nickname', '')
        game_id = player.get('gameId', '')
        season_id = player.get('seasonId', '')
        game_Rank = player.get('gameRank', '')
        character_Num = player.get('characterNum', '')
        team_kill = player.get('teamKill','')
        kill = player.get('playerKill', '')
        death = player.get('playerDeaths', '')
        assist = player.get('playerAssistant','')
        dmg_player = player.get('damageToPlayer','')
        dmg_monster = player.get('damageToMonster','')
        
        # Get masteryLevel dict
        mastery = player.get('masteryLevel', {})
        
        # Extract weapon mastery ID
        weapon_type_id = ''
        for mastery_id in mastery:
            if mastery_id in weapon_mastery_ids:
                weapon_type_id = mastery_id  # Keep as string
                break

        # Look up readable names
        character_name = character_map.get(str(character_Num), f"Unknown({character_Num})")
        weapon_name = weapon_map.get(weapon_type_id, f"Unknown({weapon_type_id})")
        
        # Get equipment data
        equipment = player.get('equipment', {})
        item_weapon = item_map.get(str(equipment.get('0', '')), 'Unknown(Weapon)')
        item_chest = item_map.get(str(equipment.get('1', '')), 'Unknown(Chest)')
        item_head = item_map.get(str(equipment.get('2', '')), 'Unknown(Head)')
        item_arm = item_map.get(str(equipment.get('3', '')), 'Unknown(Arm)')
        item_leg = item_map.get(str(equipment.get('4', '')), 'Unknown(Leg)')
        
        # === AUGMENT PARSING ===
        first_sub = player.get('traitFirstSub', [])
        second_sub = player.get('traitSecondSub', [])

        trait_main1 = augment_map.get(str(first_sub[0]), f"Unknown({first_sub[0]})") if len(first_sub) > 0 else ""
        trait_main2 = augment_map.get(str(first_sub[1]), f"Unknown({first_sub[1]})") if len(first_sub) > 1 else ""
        trait_sub1 = augment_map.get(str(second_sub[0]), f"Unknown({second_sub[0]})") if len(second_sub) > 0 else ""
        trait_sub2 = augment_map.get(str(second_sub[1]), f"Unknown({second_sub[1]})") if len(second_sub) > 1 else ""

        # Append player data to the list
        players_data.append({
            'userNum': user_num,
            'nickname': nickname,
            'gameId': game_id,
            'seasonId': season_id,
            'gameRank': game_Rank,
            'character': character_name,
            'weapon': weapon_name,
            'itemWeapon': item_weapon,
            'itemChest': item_chest,
            'itemHead': item_head,
            'itemArm': item_arm,
            'itemLeg': item_leg,
            'Trait_Main1': trait_main1,
            'Trait_Main2': trait_main2,
            'Trait_Sub1': trait_sub1,
            'Trait_Sub2': trait_sub2,
            'TeamKill' : team_kill,
            'Kill' : kill,
            'Death' : death,
            'Assist' : assist,
            'Dmg_Player' : dmg_player,
            'Dmg_Monster' : dmg_monster
        })

    return players_data

# Call the function and get the game data
game_data = get_game_data(game_id)

# Print the game data in a readable format
if game_data:
    for player in game_data:
        print(f"Player: {player['nickname']} (UserNum: {player['userNum']})")
        print(f"  Game ID: {player['gameId']} | Season ID: {player['seasonId']} | Rank: {player['gameRank']}")
        print(f"  Character: {player['character']} | Weapon: {player['weapon']}")
        print(f"  Equipment: Weapon: {player['itemWeapon']}, Chest: {player['itemChest']}, Head: {player['itemHead']}, Arm: {player['itemArm']}, Leg: {player['itemLeg']}")
        print(f"  Traits: Main1: {player['Trait_Main1']}, Main2: {player['Trait_Main2']}, Sub1: {player['Trait_Sub1']}, Sub2: {player['Trait_Sub2']}")
        print(f"  Team Kill: {player['TeamKill']} | Kills: {player['Kill']} | Deaths: {player['Death']} | Assists: {player['Assist']}")
        print(f"  Damage to Players: {player['Dmg_Player']} | Damage to Monsters: {player['Dmg_Monster']}")
        print("-" * 40)
else:
    print("No data available.")


Player: 尋風追憶 (UserNum: 5359969)
  Game ID: 46678263 | Season ID: 31 | Rank: 7
  Character: Leon | Weapon: Glove
  Equipment: Weapon: Obsidian_Jitte, Chest: Covert_Agent_Uniform, Head: Astronaut's_Helmet, Arm: Pharaoh's_Artifact, Leg: Delta_Red
  Traits: Main1: Open_Wounds, Main2: Overwatch, Sub1: Dismantle_Goliath, Sub2: Anima_Reaper
  Team Kill: 12 | Kills: 3 | Deaths: 2 | Assists: 7
  Damage to Players: 10909 | Damage to Monsters: 31758
----------------------------------------
Player: 뎁미니스트 (UserNum: 5294646)
  Game ID: 46678263 | Season ID: 31 | Rank: 5
  Character: Debi_&_Marlene | Weapon: Two-handed_Sword
  Equipment: Weapon: Aurora_Longsword, Chest: Phantom_Jacket, Head: Commander_Headset, Arm: Moonlight_Pendant, Leg: Cowboy_Boots
  Traits: Main1: Camping_Guide, Main2: Penny_Pincher, Sub1: Embolden, Sub2: Dine_n'_Dash
  Team Kill: 15 | Kills: 2 | Deaths: 1 | Assists: 11
  Damage to Players: 13044 | Damage to Monsters: 49417
----------------------------------------
Player: 범지 (Use

In [ ]:
game_id = 46678263

# Function to get the game data for all players in a game
def get_game_data(game_id):
    url = f'https://open-api.bser.io/v1/games/{game_id}'
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to fetch game data: {response.status_code} - {response.text}")
        return None

    data = response.json()

    # Initialize list to store player data
    players_data = []

    # Define valid weapon mastery IDs (as strings)
    weapon_mastery_ids = {
        "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11",
        "12", "13", "14", "15", "16", "17", "18", "19", "20",
        "21", "22", "23", "24", "25"
    }
        
    # Iterate through players in the 'userGames' list
    for player in data.get('userGames', []):
        user_num = player.get('userNum', '')
        nickname = player.get('nickname', '')
        game_id = player.get('gameId', '')
        season_id = player.get('seasonId', '')
        game_Rank = player.get('gameRank', '')
        character_Num = player.get('characterNum', '')
        team_kill = player.get('teamKill','')
        kill = player.get('playerKill', '')
        death = player.get('playerDeaths', '')
        assist = player.get('playerAssistant','')
        dmg_player = player.get('damageToPlayer','')
        dmg_monster = player.get('damageToMonster','')
        

        # Get masteryLevel dict
        mastery = player.get('masteryLevel', {})
        
        # Extract weapon mastery ID
        weapon_type_id = ''
        for mastery_id in mastery:
            if mastery_id in weapon_mastery_ids:
                weapon_type_id = mastery_id  # Keep as string
                break

        # Look up readable names
        character_name = character_map.get(str(character_Num), f"Unknown({character_Num})")
        weapon_name = weapon_map.get(weapon_type_id, f"Unknown({weapon_type_id})")
        
        # Get equipment data
        equipment = player.get('equipment', {})
        item_weapon = item_map.get(str(equipment.get('0', '')), 'Unknown(Weapon)')
        item_chest = item_map.get(str(equipment.get('1', '')), 'Unknown(Chest)')
        item_head = item_map.get(str(equipment.get('2', '')), 'Unknown(Head)')
        item_arm = item_map.get(str(equipment.get('3', '')), 'Unknown(Arm)')
        item_leg = item_map.get(str(equipment.get('4', '')), 'Unknown(Leg)')
        
        # === AUGMENT PARSING ===
        first_sub = player.get('traitFirstSub', [])
        second_sub = player.get('traitSecondSub', [])

        trait_main1 = augment_map.get(str(first_sub[0]), f"Unknown({first_sub[0]})") if len(first_sub) > 0 else ""
        trait_main2 = augment_map.get(str(first_sub[1]), f"Unknown({first_sub[1]})") if len(first_sub) > 1 else ""
        trait_sub1 = augment_map.get(str(second_sub[0]), f"Unknown({second_sub[0]})") if len(second_sub) > 0 else ""
        trait_sub2 = augment_map.get(str(second_sub[1]), f"Unknown({second_sub[1]})") if len(second_sub) > 1 else ""

        # Append player data to the list
        players_data.append({
            'userNum': user_num,
            'nickname': nickname,
            'gameId': game_id,
            'seasonId': season_id,
            'gameRank': game_Rank,
            'character': character_name,
            'weapon': weapon_name,
            'itemWeapon': item_weapon,
            'itemChest': item_chest,
            'itemHead': item_head,
            'itemArm': item_arm,
            'itemLeg': item_leg,
            'Trait_Main1': trait_main1,
            'Trait_Main2': trait_main2,
            'Trait_Sub1': trait_sub1,
            'Trait_Sub2': trait_sub2,
            'TeamKill' : team_kill,
            'Kill' : kill,
            'Death' : death,
            'Assist' : assist,
            'Dmg_Player' : dmg_player,
            'Dmg_Monster' : dmg_monster
        })

    return players_data

[{'userNum': 5359969, 'nickname': '尋風追憶', 'gameId': 46678263, 'seasonId': 31, 'gameRank': 7, 'characterNum': 29, 'weaponMasteryId': '1', 'itemWeapon': 111501, 'itemChest': 202404, 'itemHead': 201507, 'itemArm': 705608, 'itemLeg': 204419, 'Trait_Main1': 7010701, 'Trait_Main2': 7310301, 'Trait_Sub1': 7010501, 'Trait_Sub2': 7010601, 'TeamKill': 12, 'Kill': 3, 'Death': 2, 'Assist': 7, 'Dmg_Player': 10909, 'Dmg_Monster': 31758}, {'userNum': 5294646, 'nickname': '뎁미니스트', 'gameId': 46678263, 'seasonId': 31, 'gameRank': 5, 'characterNum': 65, 'weaponMasteryId': '16', 'itemWeapon': 102412, 'itemChest': 202506, 'itemHead': 201522, 'itemArm': 205401, 'itemLeg': 204417, 'Trait_Main1': 7110801, 'Trait_Main2': 7210801, 'Trait_Sub1': 7110101, 'Trait_Sub2': 7110601, 'TeamKill': 15, 'Kill': 2, 'Death': 1, 'Assist': 11, 'Dmg_Player': 13044, 'Dmg_Monster': 49417}, {'userNum': 4911433, 'nickname': '범지', 'gameId': 46678263, 'seasonId': 31, 'gameRank': 4, 'characterNum': 47, 'weaponMasteryId': '4', 'itemWea

In [39]:
print(len(players_data))


24


In [5]:
import csv
import os

def save_players_data_to_csv(players_data, output_file='players_data.csv', mode='append'):
    if not players_data:
        print("No player data to save.")
        return

    headers = players_data[0].keys()

    if mode == 'replace':
        # Replace mode: overwrite the file
        file_exists = False
    elif mode == 'append':
        # Append mode: check if the file exists
        file_exists = os.path.isfile(output_file)
    else:
        print("Invalid mode! Use 'append' or 'replace'.")
        return

    with open(output_file, 'a' if mode == 'append' else 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        if not file_exists:
            writer.writeheader()  # Write headers only if the file is being created
        writer.writerows(players_data)

    print(f"Data saved to {output_file} in {mode} mode.")


In [16]:
save_players_data_to_csv(players_data, mode='append')


NameError: name 'players_data' is not defined

In [53]:
save_players_data_to_csv(players_data, mode='replace')


Data saved to players_data.csv in replace mode.


In [ ]:
import random
import time


# Function to run the process multiple times with random game IDs within a range
def run_multiple_games(start_game_id, end_game_id, num_runs, mode='append'):
    for _ in range(num_runs):
        # Generate a random game ID within the specified range
        game_id = random.randint(start_game_id, end_game_id)
        
        # Get game data
        players_data = get_game_data(game_id)
        
        # Save to CSV
        if players_data:
            save_players_data_to_csv(players_data, mode=mode)
        
        #delay
        time.sleep(1)  #ER api is 1 request per second

# Example usage
run_multiple_games(start_game_id=47181124, end_game_id=47990108, num_runs=2, mode='append')


Data saved to players_data.csv in append mode.
Data saved to players_data.csv in append mode.
